In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.stats import norm

print("All libraries imported successfully!")


All libraries imported successfully!


In [2]:
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']
start_date = '2020-01-01'
end_date = '2023-01-01'

data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
data.head()


[*********************100%***********************]  5 of 5 completed


Ticker,AAPL,AMZN,GOOGL,MSFT,TSLA
Date,,,,,
2020-01-02 00:00:00+00:00,72.876106,94.900497,68.264961,153.938202,28.684000
2020-01-03 00:00:00+00:00,72.167618,93.748497,67.907852,152.021423,29.534000
2020-01-06 00:00:00+00:00,72.742668,95.143997,69.717865,152.414352,30.102667
2020-01-07 00:00:00+00:00,72.400536,95.343002,69.583206,151.024673,31.270666
2020-01-08 00:00:00+00:00,73.565193,94.598503,70.078476,153.430283,32.809334


In [3]:
returns = data.pct_change().dropna()
returns.head()


Ticker,AAPL,AMZN,GOOGL,MSFT,TSLA
Date,,,,,
2020-01-03 00:00:00+00:00,-0.009722,-0.012139,-0.005231,-0.012452,0.029633
2020-01-06 00:00:00+00:00,0.007968,0.014886,0.026654,0.002585,0.019255
2020-01-07 00:00:00+00:00,-0.004703,0.002092,-0.001931,-0.009118,0.038801
2020-01-08 00:00:00+00:00,0.016086,-0.007809,0.007118,0.015929,0.049205
2020-01-09 00:00:00+00:00,0.021241,0.004799,0.010498,0.012493,-0.021945


In [4]:
confidence_level = 0.95
portfolio_value = 1000000  # Example portfolio value

# Calculate portfolio returns
portfolio_returns = returns.mean(axis=1)

# Calculate VaR
var_historical = np.percentile(portfolio_returns, (1 - confidence_level) * 100)
var_historical_value = portfolio_value * var_historical

print(f"Historical VaR at {confidence_level*100}% confidence level: ${-var_historical_value:.2f}")


Historical VaR at 95.0% confidence level: $37051.03


In [5]:
mean_return = portfolio_returns.mean()
std_dev = portfolio_returns.std()

var_parametric = norm.ppf(1 - confidence_level) * std_dev - mean_return
var_parametric_value = portfolio_value * var_parametric

print(f"Parametric VaR at {confidence_level*100}% confidence level: ${-var_parametric_value:.2f}")


Parametric VaR at 95.0% confidence level: $38361.01


In [6]:
num_simulations = 10000
simulation_days = 252

simulated_returns = np.random.normal(mean_return, std_dev, (simulation_days, num_simulations))
simulated_portfolio_values = portfolio_value * (1 + simulated_returns).cumprod(axis=0)

# Calculate VaR
var_monte_carlo = np.percentile(simulated_portfolio_values[-1], (1 - confidence_level) * 100)
var_monte_carlo_value = portfolio_value - var_monte_carlo

print(f"Monte Carlo VaR at {confidence_level*100}% confidence level: ${var_monte_carlo_value:.2f}")


Monte Carlo VaR at 95.0% confidence level: $317545.72


In [7]:
# Calculate actual losses
actual_losses = portfolio_returns * portfolio_value

# Check if actual losses exceed VaR
exceedances = actual_losses < -var_historical_value
num_exceedances = exceedances.sum()

print(f"Number of exceedances for Historical VaR: {num_exceedances}")


Number of exceedances for Historical VaR: 719


In [8]:
exceedances_parametric = actual_losses < -var_parametric_value
num_exceedances_parametric = exceedances_parametric.sum()

print(f"Number of exceedances for Parametric VaR: {num_exceedances_parametric}")


Number of exceedances for Parametric VaR: 724


In [9]:
exceedances_monte_carlo = actual_losses < -var_monte_carlo_value
num_exceedances_monte_carlo = exceedances_monte_carlo.sum()

print(f"Number of exceedances for Monte Carlo VaR: {num_exceedances_monte_carlo}")


Number of exceedances for Monte Carlo VaR: 0


In [10]:
print(f"Historical VaR: ${-var_historical_value:.2f}")
print(f"Parametric VaR: ${-var_parametric_value:.2f}")
print(f"Monte Carlo VaR: ${var_monte_carlo_value:.2f}")

print(f"Exceedances (Historical): {num_exceedances}")
print(f"Exceedances (Parametric): {num_exceedances_parametric}")
print(f"Exceedances (Monte Carlo): {num_exceedances_monte_carlo}")


Historical VaR: $37051.03
Parametric VaR: $38361.01
Monte Carlo VaR: $317545.72
Exceedances (Historical): 719
Exceedances (Parametric): 724
Exceedances (Monte Carlo): 0
